# Overview of the software

## Appliance Class

The appliance class is the collection of characteristics of the appliance we are modelling


In [ ]:
class Appliance:

    def __init__(self, max_load, maintain_load, max_level, min_level, ramp_up, ramp_down):
        self.max_load = max_load
        self.maintain_load = maintain_load
        self.max_level = max_level
        self.min_level = min_level
        self.ramp_up = ramp_up
        self.ramp_down = ramp_down

## Model Class

The model class contains all the functionality of the software.

When the model is initiated the forecast data is pulled down and parsed.
It also initiate the model table which is used for the optimisation algorithm


In [ ]:
    def __init__(self, start_percentage, n, app, state):
        self.n = n
        self.start_n = start_percentage * self.n
        self.appliance = app
        self.predispatch = Predispatch().get_table('REGION_PRICE')
        self.predispatch = self.predispatch.loc[self.predispatch['REGIONID'] == state]
        self.forecast_price = self.extract_forecast_price()
        self.time_steps = self.forecast_price.index.tolist()
        self.n_time_steps = len(self.forecast_price)
        self.cost = self.establish_costs()
        self.ramp_up = int(app.ramp_up * self.n)
        self.ramp_down = int(app.ramp_down * self.n)
        self.max_level = int(self.appliance.max_level * self.n)
        self.min_level = int(self.appliance.min_level * self.n)
        self.model_table = self.init_model_table()
        self.cheapest_path = None

## The algorithm

This is where the bulk of the optimisation happens.

There are two main loops; <br>

run_model()<br>
For each time step we do the model_time_step loop <br>

model_time_step()<br>
each state which is reachable from the previous time step is assigned its own new Node which
keeps track of the cumulative cost to reach that node and the path that it took to reach that
new Node.<br>

The path chosen at each node is either the path that results in the lowest cumulative cost to reach
that node or it is either on or maintain if the node is at the lowest allowable state.

In [ ]:
class Node:
    def __init__(self, time):
        self.totalCost = float('nan')
        self.state_value = []
        self.path = []
        self.time = time

    def run_model(self):
        for i in range(1, self.n_time_steps):
            # print(self.forecast_price.index[i])
            self.model_time_step(self.forecast_price.index[i], i)


    def model_time_step(self, time, model_table_index):
        time_step = []
        for i in range(0, self.n):
            # Here the previous index depending on the ramps are calculated flooring and ceiling them to n
            ramp_up_index = i - self.ramp_up if i - self.ramp_up >= 0 else 0
            ramp_down_index = i + self.ramp_down if i + self.ramp_down < self.n - 1 else self.n - 1
            maintain_index = i


            # this is a dictionary of the costs depending on the ramp. It should default to maintain.
            cost_dict = {ramp_up_index: self.cost[time]['MaxLoadCost'],
                         ramp_down_index: 0,
                         maintain_index: self.cost[time]['MaintainCost']}

            previous_time_step = self.model_table[model_table_index - 1]

            ramp_up_cost = cost_dict[ramp_up_index] + previous_time_step[ramp_up_index].totalCost
            ramp_down_cost = cost_dict[ramp_down_index] + previous_time_step[ramp_down_index].totalCost
            maintain_cost = cost_dict[maintain_index] + previous_time_step[maintain_index].totalCost

            #Ensures that only paths that are reachable from the previous timestep are used
            ramp_up_cost = ramp_up_cost if not(np.isnan(ramp_up_cost)) else float_info.max
            ramp_down_cost = ramp_down_cost if not(np.isnan(ramp_down_cost)) else float_info.max
            maintain_cost = maintain_cost if not(np.isnan(maintain_cost)) else float_info.max

            # these conditionals deal with when the state is around the minimum level
            new_node = Node(time)
            if i < self.min_level:
                new_node.totalCost = ramp_up_cost
                new_node.state_value.append(ramp_up_index / self.n)
                new_node.state_value = previous_time_step[ramp_up_index].state_value + new_node.state_value
                new_node.path.append(1)
                new_node.path = previous_time_step[ramp_up_index].path + new_node.path
                time_step.append(new_node)
                continue
            elif i == self.min_level and (maintain_cost < ramp_up_cost and maintain_cost < ramp_down_cost):
                new_node.totalCost = maintain_cost
                new_node.state_value.append(maintain_index / self.n)
                new_node.state_value = previous_time_step[maintain_index].state_value + new_node.state_value
                new_node.path.append(0.5)
                new_node.path = previous_time_step[maintain_index].path + new_node.path
                time_step.append(new_node)
                continue


            # This block of conditionals chooses the least cost path to the node and appends
            # the current node to the growing list of nodes it has been to
            if ramp_up_cost < ramp_down_cost and ramp_up_cost < maintain_cost:
                # print("++++++++++++++++++++++++++++++++++++++++++")
                new_node.totalCost = ramp_up_cost
                new_node.state_value.append(ramp_up_index / self.n)
                new_node.state_value = previous_time_step[ramp_up_index].state_value + new_node.state_value
                new_node.path.append(1)
                new_node.path = previous_time_step[ramp_up_index].path + new_node.path
            elif ramp_down_cost < ramp_up_cost and ramp_down_cost < maintain_cost:
                new_node.totalCost = ramp_down_cost
                new_node.state_value.append(ramp_down_index / self.n)
                new_node.state_value = previous_time_step[ramp_down_index].state_value + new_node.state_value
                new_node.path.append(0)
                new_node.path = previous_time_step[ramp_down_index].path + new_node.path
            elif maintain_cost < ramp_up_cost and maintain_cost < ramp_down_cost:
                new_node.totalCost = maintain_cost
                new_node.state_value.append(maintain_index / self.n)
                new_node.state_value = previous_time_step[maintain_index].state_value + new_node.state_value
                new_node.path.append(0.5)
                new_node.path = previous_time_step[maintain_index].path + new_node.path
            # print(new_node.totalCost)
            time_step.append(new_node)
        # print(time)
        self.model_table.append(time_step)

## Extracting the lowest cost path

Extracting the lowest cost path is as simple as searching through all the nodes in the last
timestep and finding the node with the lowest cumulative cost that has a valid path back to
the start.


In [ ]:
    def get_cheapest_path(self):
        min = max_possible_number
        min_index = 0
        for i in range(0, self.n):
            mnode = self.model_table[self.n_time_steps - 1][i]
            m = mnode.totalCost
            if m < min and len(mnode.path) >= self.n_time_steps - 1:
                min_index = i
                min = m
        self.cheapest_path = self.model_table[self.n_time_steps - 1][min_index]
        return self.model_table[self.n_time_steps - 1][min_index], min_index

## Extracting lowest cost path to state value

Finding the lowest cost path to a particular state is simple also as the state_value variable
holds the current state of each node and its path to that state. <br>
Searching for the state as well as the lowest total cost and a valid path return the lowest cost
path to that state.

In [ ]:
    def get_cheapest_path_to_level(self, p_top, p_bottom):
        sub_table = []
        for i in range(0, self.n):
            time_step = self.model_table[self.n_time_steps - 1][i]
            end_index = len(time_step.state_value) - 1

            if end_index >= self.n_time_steps - 1 and (time_step.state_value[end_index] >= p_bottom and time_step.state_value[end_index] <= p_top):
                sub_table.append(time_step)

        min = float_info.max
        min_index = 0
        for i in range(0, len(sub_table) - 1):
            mnode = sub_table[i]
            if m < min and len(mnode.path) >= self.n_time_steps - 1:
                min_index = i
                min = m
        self.cheapest_path = sub_table[min_index]
        return sub_table[min_index], min_index
